# 🎯 Intelligent Meeting Analysis Workshop: Nova Omni

**The Challenge**: Teams spend 60% of their time in meetings, but only 20% of meetings have proper documentation. Important decisions, action items, and follow-ups are frequently lost, leading to:
- Missed deadlines
- Unclear responsibilities  
- Repeated discussions
- Poor project outcomes

**Your Mission**: Build an intelligent meeting analysis system that automatically:
1. Transcribes meeting recordings with speaker identification using Nova Omni
2. Extracts key insights and action items
3. Generates professional meeting minutes
4. Creates executive summaries for leadership
5. Ensures content safety and compliance

**The Solution**: Use Amazon Nova Omni's multimodal capabilities for audio transcription and intelligent analysis.

Let's build this solution step by step! 🚀

## 🛠️ Step 1: Setting Up Your AI Workshop Environment

**What we're doing**: Preparing our development environment with the latest AWS AI services.

**Why this matters**: Nova Omni requires the latest SDK version to access multimodal features.



In [ ]:
# Install the latest boto3 version for Nova Omni support
!pip install -r requirements.txt

print("✅ Environment setup complete! Ready to build intelligent meeting analysis.")

## 🔧 Step 2: Initializing Your AI Toolkit

**What we're doing**: Setting up connections to AWS AI services and storage.

**The Architecture**:
- **Nova Omni (us-west-2)**: For multimodal audio transcription and analysis
- **S3**: For secure audio storage and results

**Pro Tip**: Nova Omni is available in us-west-2 and other regions.

In [ ]:
import boto3
import json
import base64
from botocore.exceptions import ClientError
from botocore.config import Config
from IPython.display import Audio, display
from datetime import datetime

# Configuration
REGION_ID = "us-west-2"
MODEL_ID = "us.amazon.nova-2-omni-v1:0"

def get_bedrock_runtime():
    """Returns a Bedrock Runtime client."""
    config = Config(
        read_timeout=3 * 60,
        retries={"max_attempts": 1},
    )
    bedrock = boto3.client(
        service_name="bedrock-runtime",
        region_name=REGION_ID,
        config=config,
    )
    return bedrock

# Initialize S3 client for file storage
s3 = boto3.client('s3', region_name=REGION_ID)

# Get default bucket for storage
import sagemaker
session = sagemaker.Session()
default_bucket = session.default_bucket()

print(f"📦 Using S3 bucket: {default_bucket}")
print("🤖 AI services initialized successfully!")
print(f"🎵 Nova Omni ready in {REGION_ID} for audio processing")
print(f"🧠 Nova ready for intelligent analysis")

## 📁 Step 3: Preparing Your Meeting Recording

**The Scenario**: You've just received a recording from a weekly leadership meeting. The CEO wants a comprehensive analysis by end of day.

**What we're doing**: 
1. Download the meeting recording
2. Upload it to secure S3 storage
3. Preview the audio to understand what we're analyzing

**Security Note**: In production, meetings would be directly uploaded to S3 with proper encryption and access controls.

In [ ]:
# Download leadership meeting recording
file_name = 'leadership_meeting.mp3'
source_url = 'https://ws-assets-prod-iad-r-pdx-f3b3f9f1a7d6a3d0.s3.us-west-2.amazonaws.com/335119c4-e170-43ad-b55c-76fa6bc33719/podcastdemo.mp3'

print("📥 Downloading leadership meeting recording...")
!curl {source_url} --output {file_name}

# Upload to secure S3 storage
object_name = f'meetings/input/{file_name}'
s3.upload_file(file_name, default_bucket, object_name)

print(f"✅ Meeting recording uploaded to: s3://{default_bucket}/{object_name}")
print(f"🔒 Secure storage ready for AI processing")

In [ ]:
print("🎧 Leadership Meeting Recording:")
print("📅 Meeting Type: Weekly Leadership Sync")
print("👥 Expected Participants: CEO, CTO, VP Engineering, VP Sales")
print("🎯 Agenda: Product roadmap, Q4 planning, resource allocation")
print("\n🔊 Audio Preview:")

# Display audio player for preview
display(Audio(file_name))

## 🎙️ Step 4: Audio Transcription with Nova Omni

**The Magic Moment**: Watch Nova Omni transform audio into structured text with speaker identification.

**What makes this special**:
- **Multimodal Processing**: Nova Omni understands audio context better than traditional speech-to-text
- **Speaker Identification**: Automatically identifies different speakers
- **Context Awareness**: Understands business terminology and meeting dynamics

**Business Value**: Eliminates 2+ hours of manual transcription work per meeting.

In [ ]:
def transcribe_audio_with_nova(audio_file_path):
    """Transcribe audio using Nova Omni with speaker identification and timestamps."""
    
    # Read audio file as bytes
    with open(audio_file_path, "rb") as f:
        audio_bytes = f.read()
    
    # Prepare the request for Nova Omni
    request = {
        "modelId": MODEL_ID,
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "audio": {
                            "format": "mp3",
                            "source": {
                                "bytes": audio_bytes
                            }
                        }
                    },
                    {
                        "text": '''For each speaker turn segment, transcribe, assign a speaker label, start and end timestamps. 
You must follow the exact XML format shown in the example below:
<segment><transcription speaker="speaker_id" start="start_time" end="end_time">transcription_text</transcription></segment>

Please provide a complete transcription of the entire audio with speaker identification and timestamps.'''
                    }
                ],
            }
        ],
        "inferenceConfig": {"temperature": 0.5, "maxTokens": 4000},
    }

    bedrock_runtime = get_bedrock_runtime()
    
    try:
        print("🎙️ Starting transcription with Nova Omni...")
        response = bedrock_runtime.converse(**request)
        
        # Extract the transcription from the response
        transcription = response['output']['message']['content'][0]['text']
        
        print("✅ Transcription completed successfully!")
        return transcription
        
    except ClientError as err:
        print("❌ Error occurred during transcription:")
        print(err)
        if hasattr(err, "response"):
            print(json.dumps(err.response, indent=2))
        return None

print(boto3.__version__)

# Transcribe the meeting audio
transcription_result = transcribe_audio_with_nova(file_name)

if transcription_result:
    print("\n📝 Meeting Transcription:")
    print("=" * 50)
    print(transcription_result)
    print("=" * 50)

## 🧠 Step 5: Intelligent Meeting Analysis

**The Intelligence Layer**: Transform raw transcription into actionable business insights.

**What we're extracting**:
- **Key Topics**: Main discussion points and themes
- **Decisions Made**: Clear outcomes and resolutions
- **Action Items**: Who needs to do what by when
- **Sentiment Analysis**: Team dynamics and engagement levels
- **Risk Identification**: Potential blockers or concerns raised

**Executive Value**: Provides leadership with instant meeting insights without reading full transcripts.

In [ ]:
def analyze_meeting_content(transcription):
    """Analyze meeting transcription for key insights and themes."""
    
    print("🧠 Analyzing meeting content with Nova...")
    
    try:
        bedrock = get_bedrock_runtime()
        
        analysis_prompt = f"""
Please analyze this meeting transcription and provide a comprehensive analysis:

**TRANSCRIPTION:**
{transcription}

**ANALYSIS REQUIREMENTS:**
Please provide a structured analysis with the following sections:

## 📊 EXECUTIVE SUMMARY
- Brief overview of the meeting's purpose and outcomes
- Key decisions made
- Overall meeting effectiveness

## 🎯 KEY TOPICS DISCUSSED
- List the main topics covered
- Time allocation and priority level for each topic

## ✅ DECISIONS MADE
- Clear list of all decisions reached
- Decision owners and rationale

## 🚨 RISKS & CONCERNS
- Issues raised that need attention
- Potential blockers or challenges identified

## 📈 SENTIMENT ANALYSIS
- Overall team sentiment and engagement
- Areas of agreement vs. disagreement
- Energy levels and participation

## 🔄 FOLLOW-UP ITEMS
- Items requiring additional discussion
- Unresolved questions or concerns

Please provide detailed, actionable insights that would be valuable for leadership review.
"""
        
        response = bedrock.converse(
            modelId=MODEL_ID,
            messages=[
                {
                    "role": "user",
                    "content": [{"text": analysis_prompt}]
                }
            ],
            inferenceConfig={
                "maxTokens": 4000,
                "temperature": 0.3
            }
        )
        
        analysis = response['output']['message']['content'][0]['text']
        
        print("✅ Meeting analysis completed!")
        print(f"📊 Generated comprehensive analysis with actionable insights")
        
        return analysis
        
    except Exception as e:
        print(f"❌ Analysis failed: {str(e)}")
        return None

# Analyze the meeting content
if transcription_result:
    analysis_result = analyze_meeting_content(transcription_result)
    
    if analysis_result:
        print("\n" + "=" * 80)
        print("🧠 INTELLIGENT MEETING ANALYSIS")
        print("=" * 80)
        print(analysis_result)
        print("=" * 80)
    else:
        print("❌ Failed to analyze meeting content")
else:
    print("⚠️ No transcription available for analysis")

## 📋 Step 6: Professional Meeting Minutes Generation

**The Professional Touch**: Create board-ready meeting minutes that executives can immediately use.

**What makes these special**:
- **Executive Format**: Professional structure suitable for board presentations
- **Action-Oriented**: Clear ownership and deadlines
- **Compliance Ready**: Meets corporate governance standards
- **Searchable**: Structured for easy retrieval and reference

**Time Savings**: Reduces meeting documentation time from 2 hours to 2 minutes.

In [ ]:
def generate_meeting_minutes(transcription, analysis):
    """Generate professional meeting minutes from transcription and analysis."""
    
    print("📋 Generating professional meeting minutes...")
    
    try:
        bedrock = get_bedrock_runtime()
        
        minutes_prompt = f"""
Based on the meeting transcription and analysis provided, create professional meeting minutes suitable for executive review and corporate records.

**TRANSCRIPTION:**
{transcription[:2000]}...

**ANALYSIS:**
{analysis[:2000]}...

**MEETING MINUTES FORMAT:**

# MEETING MINUTES

**Meeting Date:** {datetime.now().strftime('%B %d, %Y')}
**Meeting Type:** Weekly Leadership Sync
**Duration:** [Estimated from content]
**Attendees:** [Identified from speakers]

## AGENDA ITEMS

### 1. [Topic Name]
**Discussion:** [Key points discussed]
**Decision:** [Decision made, if any]
**Action Items:** 
- [ ] [Action item] - Owner: [Name] - Due: [Date]

### 2. [Next Topic]
[Continue format...]

## DECISIONS SUMMARY
1. [Decision 1] - Rationale: [Why]
2. [Decision 2] - Rationale: [Why]

## ACTION ITEMS SUMMARY
| Action Item | Owner | Due Date | Priority |
|-------------|-------|----------|----------|
| [Item] | [Name] | [Date] | [High/Med/Low] |

## RISKS & ESCALATIONS
- [Risk 1]: [Description and mitigation plan]
- [Risk 2]: [Description and mitigation plan]

## NEXT MEETING
**Date:** [Next scheduled meeting]
**Key Topics:** [Items to be discussed]

---
*Minutes prepared by: AI Meeting Assistant*
*Distribution: Leadership Team*

Please create comprehensive, professional minutes following this format.
"""
        
        response = bedrock.converse(
            modelId=MODEL_ID,
            messages=[
                {
                    "role": "user",
                    "content": [{"text": minutes_prompt}]
                }
            ],
            inferenceConfig={
                "maxTokens": 4000,
                "temperature": 0.2
            }
        )
        
        minutes = response['output']['message']['content'][0]['text']
        
        print("✅ Professional meeting minutes generated!")
        print(f"📄 Created executive-ready documentation")
        
        return minutes
        
    except Exception as e:
        print(f"❌ Minutes generation failed: {str(e)}")
        return None

# Generate meeting minutes
if transcription_result and analysis_result:
    minutes_result = generate_meeting_minutes(transcription_result, analysis_result)
    
    if minutes_result:
        print("\n" + "=" * 80)
        print("📋 PROFESSIONAL MEETING MINUTES")
        print("=" * 80)
        print(minutes_result)
        print("=" * 80)
    else:
        print("❌ Failed to generate meeting minutes")
else:
    print("⚠️ Missing transcription or analysis for minutes generation")

## 🎯 Step 7: Action Items Extraction

**The Accountability Engine**: Extract and organize action items for maximum follow-through.

**Smart Features**:
- **Owner Identification**: Automatically assigns responsibility based on context
- **Priority Assessment**: Evaluates urgency and importance
- **Deadline Extraction**: Identifies explicit and implicit due dates
- **Dependency Mapping**: Links related action items

**Management Value**: Ensures nothing falls through the cracks and improves team accountability.

In [ ]:
def extract_action_items(transcription):
    """Extract and organize action items from meeting transcription."""
    
    print("🎯 Extracting action items and assignments...")
    
    try:
        bedrock = get_bedrock_runtime()
        
        action_prompt = f"""
Analyze this meeting transcription and extract ALL action items, commitments, and follow-up tasks.

**TRANSCRIPTION:**
{transcription}

**EXTRACTION REQUIREMENTS:**

Please identify and organize action items using this format:

# 🎯 STRATEGIC ACTION PLAN

## IMMEDIATE ACTIONS (Next 1-2 weeks)
### High Priority
- **Action:** [Specific task description]
  - **Owner:** [Person responsible - identified from context]
  - **Due Date:** [Explicit or estimated deadline]
  - **Success Criteria:** [How to measure completion]
  - **Dependencies:** [What needs to happen first]

### Medium Priority
[Same format...]

## SHORT-TERM ACTIONS (2-4 weeks)
[Same format...]

## LONG-TERM ACTIONS (1+ months)
[Same format...]

## DECISION POINTS REQUIRING FOLLOW-UP
- **Decision Needed:** [What needs to be decided]
- **Decision Maker:** [Who will decide]
- **Information Required:** [What's needed to make the decision]
- **Timeline:** [When decision is needed]

## ACCOUNTABILITY MATRIX
| Owner | Action Count | High Priority | Due This Week |
|-------|--------------|---------------|---------------|
| [Name] | [#] | [#] | [#] |

## RISK MITIGATION ACTIONS
- **Risk:** [Identified risk]
- **Mitigation Action:** [What to do]
- **Owner:** [Who's responsible]
- **Timeline:** [When to complete]

Please be thorough and extract even implied commitments and follow-up items.
"""
        
        response = bedrock.converse(
            modelId=MODEL_ID,
            messages=[
                {
                    "role": "user",
                    "content": [{"text": action_prompt}]
                }
            ],
            inferenceConfig={
                "maxTokens": 4000,
                "temperature": 0.1
            }
        )
        
        actions = response['output']['message']['content'][0]['text']
        
        print("✅ Action items extracted successfully!")
        print(f"📋 Created comprehensive action plan with accountability")
        
        return actions
        
    except Exception as e:
        print(f"❌ Action item extraction failed: {str(e)}")
        return None

# Extract action items
strategic_actions = extract_action_items(transcription_result)

if strategic_actions:
    print("\n" + "=" * 80)
    print("🎯 STRATEGIC ACTION PLAN")
    print("=" * 80)
    print(strategic_actions)
    print("=" * 80)
else:
    print("❌ Failed to extract action items")

## 🚀 Step 8: Complete Meeting Analysis Pipeline

**What we've built**: A complete end-to-end meeting analysis system using Nova Omni.

**Key Achievements**:
- ✅ **Audio Transcription**: Automated speech-to-text with speaker identification
- ✅ **Intelligent Analysis**: AI-powered extraction of key insights
- ✅ **Professional Minutes**: Business-ready meeting documentation
- ✅ **Secure Storage**: Centralized S3 storage for all outputs


**Business Impact**:
- **Time Savings**: 90% reduction in manual meeting documentation time
- **Improved Accountability**: Clear action items and assignments
- **Better Decision Making**: Comprehensive meeting insights and trends
- **Compliance**: Automated record keeping for regulatory requirements

In [ ]:
def create_meeting_pipeline(audio_file_path):
    """Complete meeting analysis pipeline function."""
    
    print("🚀 Starting Complete Meeting Analysis Pipeline")
    print("=" * 50)
    
    # Step 1: Transcription
    print("\n📝 Step 1: Audio Transcription")
    transcription = transcribe_audio_with_nova(audio_file_path)
    
    if not transcription:
        print("❌ Pipeline failed at transcription step")
        return None
    
    # Step 2: Analysis
    print("\n🧠 Step 2: Content Analysis")
    analysis = analyze_meeting_content(transcription)
    
    if not analysis:
        print("❌ Pipeline failed at analysis step")
        return None
    
    # Step 3: Minutes Generation
    print("\n📋 Step 3: Meeting Minutes Generation")
    minutes = generate_meeting_minutes(transcription, analysis)
    
    if not minutes:
        print("❌ Pipeline failed at minutes generation step")
        return None
    
    # Step 4: Save Results
    print("\n💾 Step 4: Saving Results")
    
    # Save results to S3
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    saved_files = []
    
    try:
        # Save transcription
        trans_key = f'meetings/output/{timestamp}_transcription.txt'
        s3.put_object(Bucket=default_bucket, Key=trans_key, Body=transcription)
        saved_files.append(trans_key)
        
        # Save analysis
        analysis_key = f'meetings/output/{timestamp}_analysis.txt'
        s3.put_object(Bucket=default_bucket, Key=analysis_key, Body=analysis)
        saved_files.append(analysis_key)
        
        # Save minutes
        minutes_key = f'meetings/output/{timestamp}_minutes.txt'
        s3.put_object(Bucket=default_bucket, Key=minutes_key, Body=minutes)
        saved_files.append(minutes_key)
        
        print(f"✅ Results saved to S3: {len(saved_files)} files")
        
    except Exception as e:
        print(f"❌ Error saving to S3: {e}")
    
    print("\n✅ Pipeline completed successfully!")
    print(f"📁 {len(saved_files)} files saved to S3")
    
    return {
        "transcription": transcription,
        "analysis": analysis,
        "minutes": minutes,
        "saved_files": saved_files
    }

# Example of running the complete pipeline
print("🎯 Meeting Analysis Pipeline Summary")
print("\nThis notebook demonstrates a complete meeting analysis solution using Nova Omni:")
print("\n1. 🎙️ Audio transcription with speaker identification")
print("2. 🧠 Intelligent content analysis and insights extraction")
print("3. 📋 Professional meeting minutes generation")
print("4. 💾 Secure storage and organization in S3")

# Uncomment the line below to run the complete pipeline again
# pipeline_results = create_meeting_pipeline(file_name)

## 🎉 Workshop Conclusion: Your AI-Powered Meeting Revolution

**Congratulations!** You've successfully built an enterprise-grade intelligent meeting analysis system using Amazon Nova Omni that transforms how organizations capture, analyze, and act on meeting insights.

### 🏆 What You've Accomplished

**Core Capabilities Built:**
- ✅ **Multimodal Audio Processing**: Leveraged Nova Omni's advanced speech-to-text with speaker identification
- ✅ **Intelligent Content Analysis**: AI-powered extraction of key insights, sentiment, and decision points
- ✅ **Automated Documentation**: Professional meeting minutes and action item generation
- ✅ **End-to-End Pipeline**: Complete workflow from audio input to actionable business outputs

**Technical Achievements:**
- Implemented robust error handling and validation
- Created reusable, modular functions for each processing stage
- Established secure cloud storage and retrieval patterns
- Built structured prompting for consistent AI outputs
